### 비정형 데이터 융합형 기업 예측 모델
* 뉴스나 전문가 의견및 여론 기반한 비정형 데이터 수집
* 주가 등 정형 데이터 수집
* 기업의 주가 영향 요소 예측등을 통한
* 기업의 위험관리및 투자예측 분석

### 수집된 포털 정보기반 딥크롤링
* 기존에 수집된 정보를 바탕으로 매체별 크롤링
* 다양한 서브 데이터의 정형화
* URL 기반의 다양한 크롤링 기법 사용

### 분석요소
* 제목 :
* 내용 :
* 작성일 :
* 매체 :
* 작성자 :

### 네이버 뉴스 수집

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys

In [2]:
def calcDate(txt=''):
    #print(txt)
    rdate=''
    nowd=datetime.now()
    pattern='[^\d]'
    num=int(re.sub(pattern,'',txt))
    if('일' in txt):
        res=nowd + timedelta(days=num*-1)
    elif('.' in txt):
        ds=txt.replace('.','-')
        #print(ds)
        res=ds+' 00:00:00.000000'
    else:
        res=nowd + timedelta(hours=num*-1)
    return res

In [3]:
calcDate('2021.2.1')

'2021-2-1 00:00:00.000000'

In [4]:
def makeKey(txt=''):
    txt_=re.sub('http\S*//',' ',txt)
    key=re.sub('[^a-zA-Z0-9%]','',txt_)
    return key

In [13]:
def getData(whr=''):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select nidx,ncomp,nurl from nnews '+whr
        res=cursor.execute(sql).fetchall()
    return(res)

In [14]:
allurl = getData()
allurl[:5]

[(1, '연합뉴스', 'http://yna.kr/AKR20210317071000030?did=1195m'),
 (2,
  '이데일리',
  'http://www.edaily.co.kr/news/newspath.asp?newsid=01430086628984304'),
 (3, '이투데이', 'https://www.etoday.co.kr/news/view/2005423'),
 (4, '파이낸셜뉴스', 'http://www.fnnews.com/news/202103180822274935'),
 (5, '뉴스1', 'https://www.news1.kr/articles/?4244392')]

In [7]:
def insSub(d):
    dbname='navernews.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select * from subnews where nkey="'+d[5]+'"'
        res=cursor.execute(sql).fetchall()
        if(len(res)>0):
            print("이미 존재 합니다.:"+d[1])
        else:
            sql='insert into subnews (ncate,ntitle,ncomp,ndate,nurl,nkey) values(?,?,?,?,?,?)'
            res=cursor.execute(sql,d)
            print('.',end='')
        

In [18]:
def getDonga(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div.article_title')[0]
    wtitle=title_block.select('h1.title')[0].text.strip()
    wirter=title_block.select('span.report')[0].text.replace('동아닷컴 기자','').strip()
    wday=title_block.select('span.date01')[-1].text.strip()
    article_=htmls.select('div.article_txt')[0]
    pattern='<script[^>]*>((\n|\r|.)*?)<\/script>'
    article__=re.sub(pattern, ' ', str(article_))
    pattern = '<div id = "bestnews_layer">.*'
    article__=re.sub(pattern, ' ', str(article_))
    warticle=bsp(article__,'html.parser').text
    
    return [wtitle,wirter,wday,warticle]
#     print(title,wirter,wday)
turl='https://www.donga.com/news/article/all/20210317/105923783/2'
getDonga(turl)

https://www.donga.com/news/article/all/20210317/105923783/2


['신세계 정용진 지난해 연봉 33억…코로나19 영향으로 일부 감소',
 '김혜린',
 '수정 2021-03-17 14:45',
 "\n신세계그룹 제공정용진 신세계그룹 부회장이 지난해 연봉으로 33억 6800만 원을 받았다. 신세계 그룹 총수 일가는 총 142억 원을 수령했다.17일 신세계·이마트 사업보고서에 따르면 정 부회장은 지난해 이마트에서 총 33억 6800만 원을 받았다. 급여 20억 3400만 원과 상여 13억 3400만 원이다.이마트 측은 “어려운 대내외 경영 환경 속에서도 매출액 14조 2138억 원과 영업이익 2950억 원을 달성한 점을 고려했다”고 밝혔다.\n\n          window.googletag = window.googletag || {cmd: []};\n          googletag.cmd.push(function() {\n            googletag.defineSlot('/1249652/Donga_OSV_Desktop_1x1', [1, 1], 'div-gpt-ad-1595277065244-0').addService(googletag.pubads());\n            googletag.pubads().enableSingleRequest();\n            googletag.pubads().collapseEmptyDivs();\n            googletag.enableServices();\n          });\n        \n\n\n            googletag.cmd.push(function() { googletag.display('div-gpt-ad-1595277065244-0'); });\n          \n동생 정유경 신세계 총괄사장은 급여와 상여금 각각 17억 9400만 원, 11억 6600만 원으로 총 29억 6000만 원을 수령했다.\n주요기사\n\n안철수 “김종인-오세훈 요구한 단일화 방안 수용할 것”윤석열, ‘좀스러운 남탓 정권’끝낼 그릇 될 수 

In [8]:
def getDonga(url):
    res=requests.get(url)
    src=res.content.strip()
    htmls=bsp(src,'html.parser')
    print(url)
    title_block=htmls.select('div.article_title')
    if(len(tgts)==0):
        print('데이터가 없습니다.'+url)
        sys.exit(1)
    for tgt in tgts:
        mainarti=tgt.select('div.news_wrap')
        subartis=tgt.select('ul.list_cluster > li.sub_bx')
        if(len(mainarti)>0):
            title_= mainarti[0].select('a.news_tit')[0].text
            title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
            comp=mainarti[0].select('.info.press')[0].text.replace('언론사 선정','')
            date_=mainarti[0].select('span.info')[0].text.replace('시간 전','')
            date=calcDate(date_)
            url=mainarti[0].select('a.news_tit')[0]['href']
            key=makeKey(url)
            opt=1
            d=(opt,title,comp,date,url,key)
            insMain(d)
            for subarti in subartis:
                title_= subarti.select('a.sub_tit')[0].text
                title=re.sub('[^ㄱ-힣0-9 a-zA-Z]',' ',title_)
                comp=subarti.select('.sub_txt.press')[0].text.replace('언론사 선정','')
                date_=subarti.select('span.sub_txt')[0].text.replace('시간 전','').replace('네이버뉴스','')
                date=calcDate(date_)
                url=subarti.select('a.sub_tit')[0]['href']
                key=makeKey(url)
                opt=2
                d=(opt,title,comp,date,url,key)
                insMain(d)



In [20]:
allurl = getData()
allurl[:5]

[(1, '연합뉴스', 'http://yna.kr/AKR20210317071000030?did=1195m'),
 (2,
  '이데일리',
  'http://www.edaily.co.kr/news/newspath.asp?newsid=01430086628984304'),
 (3, '이투데이', 'https://www.etoday.co.kr/news/view/2005423'),
 (4, '파이낸셜뉴스', 'http://www.fnnews.com/news/202103180822274935'),
 (5, '뉴스1', 'https://www.news1.kr/articles/?4244392')]

In [23]:
for nidx,ncomp,turl in allurl[:3]:
    print('-'*50)
    #print(nidx,ncomp,turl)
    if ncomp=="동아일보":
        wdata=getDonga(turl)
        wdata.append(turl)
        wdata.append(ncomp)
        wdata.append(nidx)
    elif ncomp=="아시아경제":
        wdata=getAsia(turl)
        wdata.append(turl)
        wdata.append(ncomp)
        wdata.append(nidx)
        print(wdata)

--------------------------------------------------
--------------------------------------------------


NameError: name 'getEdaily' is not defined